In [ ]:
import matlab
import matlab.engine
import numpy as np
import tensorflow as tf
import keras
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Lambda

eng = matlab.engine.start_matlab()

In [ ]:
np.random.seed(7)
X_train = np.loadtxt( "mu_train.txt", delimiter = "\n" )
Y_train = np.loadtxt( "exact_sol_train.txt", delimiter = "\t" )
print(X_train)
print(Y_train)
# print(len(Y_train))
# print(len(Y_train[0]))

model = Sequential()
model.add( Dense( 12, input_dim = 1, activation = "relu" ) )
model.add( Dense( 8, activation = "relu" ) )
model.add( Dense( 1, activation = "sigmoid" ) )
model.add( Lambda( lambda x: x*2 ) )

def my_lambda_layer(x):
    y = x
    print(x)
    for i in range(80):
        y = K.concatenate( [y, x], axis=1 )
        # print(y.shape)
    return y

def my_lambda_layer_output_shape(input_shape):
    return (input_shape[0], 81)

model.add( Lambda( my_lambda_layer, output_shape = my_lambda_layer_output_shape ) )

model.summary()

model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'] )

model.fit( X_train, Y_train, epochs = 100, batch_size = 10 )